In [30]:
import pandas as pd
import numpy as np
from datasets import Dataset, ClassLabel, Value, Features
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import DataLoader
from time import time
import matplotlib.pyplot as plt

In [2]:
from sys import path
path.append('../')  
from scripts.Pipeline import Pipeline

In [3]:
Pipeline.seed_everything(42)

model_name = 'dicta-il/dictabert'

dataset = load_dataset('hebrew_sentiment')

train_size = 150
train = dataset['train'].shuffle(seed=42).select(range(train_size))
test = dataset['test']



No config specified, defaulting to: hebrew_sentiment/token
Found cached dataset hebrew_sentiment (C:/Users/Administrator/.cache/huggingface/datasets/hebrew_sentiment/token/1.0.0/2ea2ba30503e863156a06ad2d4011a75b45556d1d557c2247f6194828849f5f9)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at C:\Users\Administrator\.cache\huggingface\datasets\hebrew_sentiment\token\1.0.0\2ea2ba30503e863156a06ad2d4011a75b45556d1d557c2247f6194828849f5f9\cache-7fafd9941388ff1c.arrow


In [18]:
# load generated_tweets
generated_tweets = pd.read_csv('../data/generated_tweets.csv')

generated_tweets.head()



,text,label
0,גאה בנשיאותו של ריבלין! 🇮🇱 #נשיאות #ישראל,0
1,שיהיה לנו נשיא חכם ומשמעותי למדינת ישראל #כבוד,0
2,החלומות מתגשמים עם נשיא כמו ריבלין 🌟 #התמיכה_ה...,0
3,השנה הולך להיות מרגשת עם הנשיא החדש! #התקווה,0
4,אני מאוד משוחרר שריבלין לא יהיה יותר נשיא #שחרור,1


In [31]:
# Define the features of your generated dataset to include the ClassLabel type for the 'label' column
features = Features({
    'text': Value('string'),
    # Adjust the 'names' parameter based on the classes in your original dataset
    'label': ClassLabel(names=['pos', 'neg', 'off-topic']),
})

# Convert the pandas DataFrame to a Hugging Face Dataset with the defined features
generated_tweets_dataset = Dataset.from_pandas(generated_tweets, features=features)

# Now you can concatenate your datasets
extended_train = concatenate_datasets([train, generated_tweets_dataset.shuffle(seed=42)])

c:\Users\Administrator\miniconda3\envs\nnenv\lib\site-packages\datasets\table.py:1381: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
c:\Users\Administrator\miniconda3\envs\nnenv\lib\site-packages\datasets\table.py:1407: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
accuracies = []
times = []
training_sizes = []

# train the model with a growning number of generated tweets
for i in range(1, 11):
    print(f'Iteration {i} of 10')
    print(f'Training size: {150 + i * 300}')
    extended_train = concatenate_datasets([train, generated_tweets_dataset.shuffle(seed=42).select(range(i * 300))])
    model_name = 'dicta-il/dictabert'
    pipeline = Pipeline(model_name)
    accuracy, model, time_elapsed = pipeline.run(extended_train, test)
    print(f'Accuracy: {accuracy}')
    print(f'Time elapsed: {time_elapsed}')
    accuracies.append(accuracy)
    times.append(time_elapsed)
    training_sizes.append(150 + i * 300)
    print('---')
    method = f'generated_{i * 300}'
    model_path = f'/models/{method}.pt'
    pipeline.save_model(model_path) 

# save results to csv
results = pd.DataFrame({'accuracy': accuracies, 'time': times, 'training_size': training_sizes})
results.to_csv('../experiments/generated_results.csv', index=False)



In [ ]:
# plot the results
plt.plot(training_sizes, accuracies)
plt.xlabel('Training size')
plt.ylabel('Accuracy')
plt.title('Accuracy as a function of training size')
plt.show()

# plot the time acccuracy trade off
plt.plot(times, accuracies)
plt.xlabel('Time elapsed')
plt.ylabel('Accuracy')
plt.title('Accuracy as a function of time elapsed')
plt.show()